In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import random

In [2]:
import glob
import os
print(os.listdir("1-4sessions/0-free"))

['0.csv', '00.csv', '000.csv', '0000.csv']


In [3]:
def read_emg(path):
    sessions_csv = []
    for path, _, files in os.walk(path):
        for name in files:
            sessions_csv.append(os.path.join(path, name))

    data = pd.concat([pd.read_csv(file, header = None) for file in sessions_csv]).values
    print('input shape', data.shape)
    
    # reshape data
    # one column - one channel
    data_x = data[:,:-1]
    data_y = data[:,-1]
    data_x = data_x.reshape((-1, 8))
    data_y = data_y.repeat(8)
    data_y = data_y.reshape((-1,1))
    data = np.concatenate((data_x, data_y), axis=1)
    print('All sessions shape: ', data.shape)

    return data

In [4]:
def emg_windowing(data, window_size):
    data_x = data[:,:-1]
    data_y = data[:,-1]
    n, m = data_x.shape
#     print('shape: ', n, m)
    size = n * m
#     print('size: ', size)
    residual_rows_num =  n % window_size
#     print('delete ', residual_rows_num, 'rows')
    if residual_rows_num != 0:
        data_x = data_x[:-residual_rows_num,:]
        data_y = data_y[:-residual_rows_num]
#     print('data_x: ', data_x.shape)
#     print('data_y: ', data_y.shape)
    data_x = data_x.reshape((-1, m * window_size))
    
    data_y = data_y.reshape((-1, window_size))
    data_y = np.array(list(map(np.mean, data_y)))
#     print('data_x: ', data_x.shape)
#     print('data_y: ', data_y.shape)
    
    mixed_classes_idxs = np.where(data_y % 1 != 0)
#     print('mixed_classes_idxs: ', mixed_classes_idxs)
#     print(mixed_classes_idxs[0])
    
    data = np.c_[data_x, data_y]
    data = np.delete(data, mixed_classes_idxs, 0)
    
    return data

In [5]:
import math

def integrated_absolute_value(segment):
    return sum([abs(s) for s in segment])

def mean_absolute_value(segment):
    return sum([abs(s) for s in segment])/len(segment)

def waveform_length(segment):
    n = len(segment)
    wl = 0
    for i in range(1, n):
        wl += abs(segment[i] - segment[i-1])
    return wl

def zero_crossing(segment):
    n = len(segment)
    zc = 0
    for i in range(n - 1):
        if segment[i] * segment[i+1] < 0:
            zc += 1
    return zc

def slope_sign_changes(segment):
    n = len(segment)
    ssc = 0
    for i in range(1, n-1):
        if segment[i-1] < segment[i] and segment[i] > segment[i+1] or segment[i-1] > segment[i] and segment[i] < segment[i+1]:
            ssc += 1
    return ssc

def root_mean_square(segment):
    return math.sqrt(sum([s*s for s in segment])/len(segment))

In [6]:
from nitime.algorithms.autoregressive import AR_est_LD
from sklearn.preprocessing import StandardScaler

def autoregression_coefficients(emg, order):
    coef = AR_est_LD(emg, order=order)[0]
    return coef

In [7]:
def calculate_features(data_x, channels_num):
    n, m = data_x.shape
    features = []
    
    for channel in range(channels_num):
        channel_features = []
        
        # Calculate MAV, ZC, SSC, WL features
        channel_features.append(list(map(mean_absolute_value, data_x[:,channel::channels_num])))
        channel_features.append(list(map(waveform_length, data_x[:,channel::channels_num])))
        channel_features.append(list(map(zero_crossing, data_x[:,channel::channels_num])))
        channel_features.append(list(map(slope_sign_changes, data_x[:,channel::channels_num])))
        
        # calculate AR6 coefficients
        ar_order = 6
        ar_coef = np.array(list(map(lambda x: autoregression_coefficients(x, ar_order), data_x[:,channel::channels_num])))
        channel_features += ar_coef.transpose().tolist()
        features += channel_features
    
    return np.array(features).transpose()

In [14]:
# prepare data
random.seed(101)
data_train = read_emg('1-4sessions')
emg_train_windows = emg_windowing(data_train, 40)

data_test = read_emg('5session')
emg_test_windows = emg_windowing(data_test, 40)

data_train_x = emg_train_windows[:,:-1]
data_train_y = emg_train_windows[:,-1]
features_train = calculate_features(data_train_x, 8)

data_test_x = emg_test_windows[:,:-1]
data_test_y = emg_test_windows[:,-1]
features_test = calculate_features(data_test_y, 8)

print('Features train shape:', features_train.shape)
print('Features test shape:', features_test.shape)

input shape (4788, 65)
All sessions shape:  (38304, 9)
input shape (1214, 65)
All sessions shape:  (9712, 9)
Features train shape: (954, 80)
Features test shape: (239, 80)


In [15]:
# train_x, test_x, train_y, test_y = train_test_split(features, data_y, test_size=0.3)

# Create svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(features_train, data_train_y)
pred_y = clf.predict(features_test)

print("Accuracy:", metrics.accuracy_score(data_test_y, pred_y))

Accuracy: 0.9330543933054394
